# keras conversion code

SNN 모델을 저장하되, IF 레이어는 ReLU로 바꿔서 저장하도록 한다. 대신 IF 레이어의 설정값은 따로 찍어서 보내기.

**주의점 : keras version 2.11이 들어간 환경에서 실행할 것! (KIST의 Neu+ 칩 구동환경과 일치시켜야 함)**

In [3]:
# path = '/data/hongwonseok/ECG-SNN/KIST/MLP/MLP_binary_NO_BatchNorm.pth'

path = '/home/hschoi/data/leehyunwon/ECG-SNN/SNN_MLP_ver5_LIF_poisson_positive_only_binary_encoders187_hidden1000_encoderGradTrue_early30_lr0.001_threshold1.0_2024_11_05_19_13_23_lastEpoch.pt'
 
 

In [4]:
import torch

model = torch.load(path)
model

/tmp/ipykernel_2346339/3090495766.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(path)


{'epoch': 149,
 'model_state_dict': OrderedDict([('hidden.0.weight',
               tensor([[1.7970e-03, 7.6708e-04, 7.6870e-04,  ..., 3.4326e+00, 4.0408e+00,
                        3.4362e+00],
                       [3.7384e-01, 9.7579e-03, 1.4517e-03,  ..., 8.2034e-01, 4.5643e-01,
                        5.7082e-01],
                       [1.3272e-03, 1.6771e-03, 1.4202e-03,  ..., 7.4936e-01, 2.2146e-02,
                        5.5253e-02],
                       ...,
                       [4.0657e-04, 8.1078e-04, 1.3806e-03,  ..., 3.6654e+00, 4.7772e+00,
                        3.3208e+00],
                       [7.6067e-04, 8.0444e-05, 7.6138e-03,  ..., 3.0507e+00, 3.7444e+00,
                        3.0639e-01],
                       [8.2957e-02, 1.2664e+00, 2.3958e-03,  ..., 2.1972e-02, 1.0780e-02,
                        2.6476e-02]], device='cuda:0')),
              ('layer.0.weight',
               tensor([[-0.0040, -0.0004, -0.0002,  ..., -0.0038, -0.0036,  0.0147],
   

In [4]:
weight = model['model_state_dict']
weight

OrderedDict([('hidden.0.weight',
              tensor([[-4.7883e-01, -2.1000e+00, -1.3955e+00,  ...,  6.5632e-01,
                        5.5826e-01,  1.0480e+00],
                      [-1.0085e+00, -2.1185e+00, -5.4350e-01,  ...,  2.2340e+00,
                        1.1446e+00,  3.1596e+00],
                      [ 6.9557e-01,  5.1900e-01,  6.8710e-01,  ..., -1.0271e+00,
                       -8.2129e-01, -1.1322e+00],
                      ...,
                      [ 2.9812e-01,  6.3216e-01, -5.2330e-01,  ...,  5.0113e-02,
                       -4.6645e-01, -2.7735e-01],
                      [-2.8222e-01, -1.3380e+00, -9.7336e-02,  ...,  4.3523e-01,
                        4.6908e-01,  6.8050e-01],
                      [-4.7677e-01, -1.3803e+00,  4.4510e-01,  ...,  2.7520e-03,
                        2.1900e-01, -3.9930e-01]], device='cuda:0')),
             ('layer.0.weight',
              tensor([[-0.5208, -0.5810,  0.3991,  ...,  0.3393, -0.6402,  0.1393],
                  

In [5]:
weight.items()

odict_items([('hidden.0.weight', tensor([[-4.7883e-01, -2.1000e+00, -1.3955e+00,  ...,  6.5632e-01,
          5.5826e-01,  1.0480e+00],
        [-1.0085e+00, -2.1185e+00, -5.4350e-01,  ...,  2.2340e+00,
          1.1446e+00,  3.1596e+00],
        [ 6.9557e-01,  5.1900e-01,  6.8710e-01,  ..., -1.0271e+00,
         -8.2129e-01, -1.1322e+00],
        ...,
        [ 2.9812e-01,  6.3216e-01, -5.2330e-01,  ...,  5.0113e-02,
         -4.6645e-01, -2.7735e-01],
        [-2.8222e-01, -1.3380e+00, -9.7336e-02,  ...,  4.3523e-01,
          4.6908e-01,  6.8050e-01],
        [-4.7677e-01, -1.3803e+00,  4.4510e-01,  ...,  2.7520e-03,
          2.1900e-01, -3.9930e-01]], device='cuda:0')), ('layer.0.weight', tensor([[-0.5208, -0.5810,  0.3991,  ...,  0.3393, -0.6402,  0.1393],
        [ 0.4459,  0.6468, -0.4045,  ..., -0.2010,  0.8286, -0.1696]],
       device='cuda:0'))])

In [8]:
import torch
import torch.nn as nn
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, ReLU, Dropout, InputLayer
from tensorflow.keras.layers import ZeroPadding1D, Conv1D, BatchNormalization, ReLU, AveragePooling1D, Flatten, Dense

일단 GPT에게 변환 코드 예시를 받아놨으니 그걸로 진행해보자.

**KIST용으로 바꿀 땐 hidden layer 뉴런 수가 1000개 임에 주의!!!!**

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, ReLU

def create_keras_model_SNN(num_classes, num_encoders, hidden_size, hidden_size_2):
    # 입력 레이어 정의
    inputs = Input(shape=(num_encoders,), name='input')
    
    # 첫 번째 히든 레이어
    x = Dense(hidden_size, use_bias=False, name='hidden1')(inputs)
    x = ReLU(name='relu1')(x)
    
    # 히든레이어 하나만 써서 학습하는 모델로 변경
    # # 두 번째 히든 레이어
    # x = Dense(hidden_size_2, use_bias=False, name='hidden2')(x)
    # x = ReLU(name='relu2')(x)
    
    # 출력 레이어
    predictions = Dense(num_classes, use_bias=False, name='output')(x)
    predictions = ReLU(name='output_relu')(predictions)  # IFNode를 ReLU로 치환

    # 모델 생성
    model = Model(inputs=inputs, outputs=predictions)
    
    return model

# 모델 생성 예시
keras_model = create_keras_model_SNN(num_classes=2, num_encoders=187, hidden_size=1000, hidden_size_2=1000)

# 모델 구조 확인
keras_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 187)]             0         
                                                                 
 hidden1 (Dense)             (None, 1000)              187000    
                                                                 
 relu1 (ReLU)                (None, 1000)              0         
                                                                 
 output (Dense)              (None, 2)                 2000      
                                                                 
 output_relu (ReLU)          (None, 2)                 0         
                                                                 
Total params: 189,000
Trainable params: 189,000
Non-trainable params: 0
_________________________________________________________________


2024-10-30 17:32:27.023441: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-30 17:32:27.025701: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [11]:
# keras 모델의 초기 가중치 확인
for layer in keras_model.layers:
    weights = layer.get_weights()
    print(f"Layer: {layer.name}")
    for temp_weight in weights:
        print(temp_weight)

Layer: input
Layer: hidden1
[[-0.06366645 -0.01185206  0.00050618 ...  0.02719269 -0.02091796
  -0.03447803]
 [ 0.06094223 -0.03147082  0.0093798  ...  0.0534135   0.0507236
  -0.02673078]
 [-0.02486161 -0.02915781 -0.03580278 ... -0.01987555  0.04763599
  -0.05257952]
 ...
 [-0.06266032  0.06343159  0.06716911 ...  0.02507526  0.0092084
  -0.01930699]
 [-0.02473257  0.04427122  0.05919512 ...  0.070154   -0.044639
   0.03611359]
 [-0.00754567 -0.03398001  0.04247361 ... -0.03664387 -0.0056309
   0.00074009]]
Layer: relu1
Layer: output
[[ 0.07676843 -0.07009397]
 [ 0.06270686  0.06770964]
 [-0.03963278  0.04665702]
 ...
 [ 0.07335906  0.06514953]
 [-0.02309464 -0.01582494]
 [-0.03710136 -0.0680869 ]]
Layer: output_relu


가중치 추출하고 케라스 모델에 대입하기

왜인진 모르겠지만 cuda 커널 이미지가 안맞는다 하다가 또 된다고 하다가 그런다. 일단 시도해서 될 때 계속 진행하도록 한다.

In [12]:
import torch
print(torch.__version__)  # PyTorch 버전 확인
print(torch.version.cuda)  # CUDA 버전 확인
print(torch.cuda.is_available())  # CUDA 사용 가능 여부 확인

!export CUDA_LAUNCH_BLOCKING=1

# PyTorch에서 가중치 추출 및 Keras에 적용
pt_weights = {k: v.cpu().numpy() for k, v in weight.items()}

# Keras 가중치 설정
# pytorch의 모델 가중치는 (출력, 입력) 순서로 저장되고 케라스는 그 반대이므로 Transpose 필요
keras_model.get_layer('hidden1').set_weights([pt_weights['hidden.0.weight'].T]) # bias 빼기!
# keras_model.get_layer('hidden2').set_weights([pt_weights['hidden2.0.weight'].T]) # 레이어 1개만 사용하는 경우 빼야 함!
keras_model.get_layer('output').set_weights([pt_weights['layer.0.weight'].T])

# 잘 들어갔나 확인?
keras_model.summary()

2.4.1
11.8
True
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 187)]             0         
                                                                 
 hidden1 (Dense)             (None, 1000)              187000    
                                                                 
 relu1 (ReLU)                (None, 1000)              0         
                                                                 
 output (Dense)              (None, 2)                 2000      
                                                                 
 output_relu (ReLU)          (None, 2)                 0         
                                                                 
Total params: 189,000
Trainable params: 189,000
Non-trainable params: 0
_________________________________________________________________


In [13]:
# 모델 저장
keras_model.save('burst_1000_LIF.h5')

In [14]:
# 모델의 가중치 확인
for layer in keras_model.layers:
    weights = layer.get_weights()
    print(f"Layer: {layer.name}")
    for temp_weight in weights:
        print(temp_weight)

Layer: input
Layer: hidden1
[[-4.7883126e-01 -1.0085399e+00  6.9557190e-01 ...  2.9812288e-01
  -2.8222498e-01 -4.7676760e-01]
 [-2.1000419e+00 -2.1185455e+00  5.1899660e-01 ...  6.3216054e-01
  -1.3379674e+00 -1.3802923e+00]
 [-1.3955481e+00 -5.4350328e-01  6.8710476e-01 ... -5.2329820e-01
  -9.7336426e-02  4.4509730e-01]
 ...
 [ 6.5631783e-01  2.2339928e+00 -1.0271481e+00 ...  5.0112635e-02
   4.3523338e-01  2.7520042e-03]
 [ 5.5825573e-01  1.1445986e+00 -8.2128531e-01 ... -4.6644902e-01
   4.6908402e-01  2.1900421e-01]
 [ 1.0479628e+00  3.1595964e+00 -1.1322427e+00 ... -2.7734637e-01
   6.8049937e-01 -3.9929610e-01]]
Layer: relu1
Layer: output
[[-0.520808    0.44594646]
 [-0.5810289   0.6468107 ]
 [ 0.3990503  -0.4045053 ]
 ...
 [ 0.33933774 -0.20098811]
 [-0.64021105  0.8286197 ]
 [ 0.13931262 -0.16964607]]
Layer: output_relu


In [15]:
from tensorflow.keras.models import load_model

# 저장된 모델 로드
loaded_model = load_model('/home/hschoi/leehyunwon/ECG-SNN/to_keras_conversion/burst_1000_LIF.h5')


In [16]:
loaded_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 187)]             0         
                                                                 
 hidden1 (Dense)             (None, 1000)              187000    
                                                                 
 relu1 (ReLU)                (None, 1000)              0         
                                                                 
 output (Dense)              (None, 2)                 2000      
                                                                 
 output_relu (ReLU)          (None, 2)                 0         
                                                                 
Total params: 189,000
Trainable params: 189,000
Non-trainable params: 0
_________________________________________________________________


In [17]:
# 로드된 모델의 가중치 확인
for layer in loaded_model.layers:
    weights = layer.get_weights()
    print(f"Layer: {layer.name}")
    for temp_weight in weights:
        print(temp_weight)


Layer: input
Layer: hidden1
[[-4.7883126e-01 -1.0085399e+00  6.9557190e-01 ...  2.9812288e-01
  -2.8222498e-01 -4.7676760e-01]
 [-2.1000419e+00 -2.1185455e+00  5.1899660e-01 ...  6.3216054e-01
  -1.3379674e+00 -1.3802923e+00]
 [-1.3955481e+00 -5.4350328e-01  6.8710476e-01 ... -5.2329820e-01
  -9.7336426e-02  4.4509730e-01]
 ...
 [ 6.5631783e-01  2.2339928e+00 -1.0271481e+00 ...  5.0112635e-02
   4.3523338e-01  2.7520042e-03]
 [ 5.5825573e-01  1.1445986e+00 -8.2128531e-01 ... -4.6644902e-01
   4.6908402e-01  2.1900421e-01]
 [ 1.0479628e+00  3.1595964e+00 -1.1322427e+00 ... -2.7734637e-01
   6.8049937e-01 -3.9929610e-01]]
Layer: relu1
Layer: output
[[-0.520808    0.44594646]
 [-0.5810289   0.6468107 ]
 [ 0.3990503  -0.4045053 ]
 ...
 [ 0.33933774 -0.20098811]
 [-0.64021105  0.8286197 ]
 [ 0.13931262 -0.16964607]]
Layer: output_relu
